In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install gensim

READ FILE AND ENGINEERRING FEATURE

In [3]:
import numpy as np
import gensim as gs
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import keras.models
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
import re
import csv




Using TensorFlow backend.


In [0]:
def pre_process(text):
    text = text.lower()
    # remove tags
    text = re.sub("<!--?.*?-->", "", text)
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    return text

In [0]:

def read_data(filename):
    file = []
    with open(filename,encoding="utf8") as fp:
        line = fp.readline()
        line =  pre_process(line)
        line = gs.utils.simple_preprocess(line)


        while line:
           file.append(line)
           line = fp.readline()
           line =  pre_process(line)
           line = gs.utils.simple_preprocess(line)
    file = np.reshape(file, (len(file),1))
    fp.close()
    return file

def load_train_data(filepath, minidx, maxidx):
    filename = filepath + str(minidx) +'.txt'
    file = read_data(filename)
    data = np.array(file)
    for i in range(minidx +1 , maxidx +1 ):
        filename = filepath + str(i) +'.txt'
        file = read_data(filename)
        data = np.concatenate((data,file), axis=0)

    labels = []
    for i in range(minidx, maxidx +1):
        labels += [i]*2000

    return data, labels


def load_test_data(filepath):
    filename_1 = filepath +  'data.txt'
    file = read_data(filename_1)
    data = np.array(file)

    filename_2 = filepath + 'label.txt'
    with open(filename_2) as f:
        labels = []
        for line in f:
            labels.append(int(line))

    f.close()
    labels = np.array(labels)
    return data, labels
  
def one_hot_coding(Y,num_class):
    Y_one_hot_code = np.zeros((len(Y),num_class))
    for i in np.arange(len(Y)):
        Y_one_hot_code[i][Y[i]-1] = 1
    return Y_one_hot_code    

USING COUNT VECTORIZER

In [0]:
def feature_engineering(data_train, data_test):
    X_train = []
    for i in np.arange(data_train.shape[0]):
        data_i = data_train[i]
        doc_i = ''
        for i in np.arange(len(data_i[0])):
            doc_i += data_i[0][i]
            doc_i += ' '
        X_train.append(doc_i)

    X_test = []
    for i in np.arange(data_test.shape[0]):
        data_i = data_test[i]
        doc_i = ''
        for i in np.arange(len(data_i[0])):
            doc_i += data_i[0][i]
            doc_i += ' '
        X_test.append(doc_i)

    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    count_vect.fit(X_train)
    X_train = count_vect.transform(X_train)
    X_test  = count_vect.transform(X_test)
    return X_train, X_test 

In [0]:
data_train, labels_train = load_train_data('/content/drive/My Drive/Intership/classify_data/train/', 1, 13)
data_test, labels_test = load_test_data('/content/drive/My Drive/Intership/classify_data/test/')
X_train, X_test = feature_engineering(data_train,data_test)

# print(data_train)
print(X_train.shape)
print(X_test.shape)

(26000, 79775)
(6500, 79775)


In [0]:
with open('people.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(lines)

writeFile.close()

CLASSIFICATION WITH DNN

In [0]:
def baseline_model(X, y ):
  input_dim = X.shape[1]
  print(input_dim)
  model = Sequential()
  model.add(Dense(units=1024, activation='relu', input_dim=input_dim, name ="Input"))
  model.add(Dropout(0.3))
  model.add(Dense(units=512, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(units=13, activation='softmax'))
	# Compile model
  model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 
  model.fit(X, y, epochs = 10, batch_size = 1000)

  return model


In [0]:
if __name__  == '__main__':
    labels_train = np.array(labels_train)
    labels_test   = np.array(labels_test)
    num_class = 13
    labels_train = one_hot_coding(labels_train, num_class)
    labels_test  = one_hot_coding(labels_test, num_class)
    print(labels_train)
    model = baseline_model(X_train, labels_train)
    scores = model.evaluate(X_test, labels_test)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
79775
Epoch 1/10
26000/26000 [==============================] - 14s 556us/step - loss: 0.7897 - acc: 0.7857
Epoch 2/10
26000/26000 [==============================] - 13s 482us/step - loss: 0.2225 - acc: 0.9388
Epoch 3/10
26000/26000 [==============================] - 13s 482us/step - loss: 0.1173 - acc: 0.9701
Epoch 4/10
26000/26000 [==============================] - 13s 484us/step - loss: 0.0869 - acc: 0.9786
Epoch 5/10
26000/26000 [==============================] - 13s 484us/step - loss: 0.0747 - acc: 0.9820
Epoch 6/10
26000/26000 [==============================] - 13s 485us/step - loss: 0.0647 - acc: 0.9842
Epoch 7/10
26000/26000 [==============================] - 13s 487us/step - loss: 0.0616 - acc: 0.9858
Epoch 8/10
26000/26000 [==============================] - 13s 483us/step - loss: 0.0570 - acc: 0.9867
Epoch 9/10
26000/26000

IMPLEMENT SVM WITH LIBRARY

In [0]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, labels_train)
print('Best score for training data:', svm_model.best_score_,"\n") 
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")


Best score for training data: 0.8372692307692308 

Best C: 100 

Best Kernel: rbf 

Best Gamma: 0.0001 



In [0]:
if __name__  == '__main__':
    y_train = np.array(labels_train)
    y_test   = np.array(labels_test)
    num_class = 13
#     labels_train = one_hot_coding(labels_train, num_class)
#     labels_test  = one_hot_coding(labels_test, num_class)
    svm_model = SVC(kernel="rbf",gamma = 0.0001,  C=100).fit(X_train, y_train)
    score = svm_model.score(X_test, y_test)
    print("\n %.2f%%" % (score*100))


 87.62%


IMPLEMENT SVM WITH NUMPY


In [0]:
def  computed_dW(W,X,y,reg):
    d, C = W.shape;
    N = X.shape[0];
    loss = 0;
    dW = np.zeros((d,C))
    id0 = np.arange(N);
    Z = X.dot(W);
    correct_core =  Z[id0,y].reshape(N,1);
    margins = np.maximum(0, Z - correct_core +1);
    margins[id0,y] = 0;
    loss = np.sum(margins);
    loss /= N;
    loss += 0.5*reg*np.sum(W*W);

    F = (margins>0).astype(int);
    id1 = np.arange(F.shape[0])
    F[id1,y] = np.sum(-F,axis=1);
    dW = X.T.dot(F)/N + reg*W;
    return loss, dW

In [0]:
def classify_svm(X,y, W, reg,lr =0.1, batch_size = 1000, epochs = 10):
    W =W;
    num_iters = int(X.shape[0]/batch_size)
    for epoch in np.arange(epochs):
      loss_history= [];
      print("Epochs : {}".format(epoch + 1))
      X, y = shuffer_data(X,y)
      for it in np.arange(num_iters):
          start = it*batch_size
          end = (it+1)*batch_size
          X_batch = X[start:end];
          y_batch = y[start:end];
          loss, dW = computed_dW(W,X_batch,y_batch,reg)
          loss_history.append(loss);
          W -= lr*dW;
          if it% 200 ==0 :
              print('it %d/%d, loss = %f' %(it, num_iters, loss_history[it]));
    return W, loss_history



In [0]:
def shuffer_data(X,y):
    N = X.shape[0]
    permutation = np.random.permutation(N)
    x_shuffer = X[permutation]
    y_shuffer = y[permutation]
    return x_shuffer, y_shuffer

In [0]:
def predict(W,X):
    Z = X.dot(W);
    return np.argmax(Z,axis=1);

def evaluate(W, X, y):
    y_pred = predict(W,X);
    acc = 100*np.mean(y_pred == y);
    return acc

In [0]:
if __name__  == '__main__':
    y_train = np.array(labels_train)
    y_test   = np.array(labels_test)
    print(y_train.shape)
    y_train -= 1
    y_test   -= 1
    num_class = 13
  
    W_init = np.ones((X_train.shape[1],num_class))
    W, loss = classify_svm(X_train ,y_train, W_init, reg = 0.01 ,lr = 0.1, batch_size = 100, epochs =5)
    acc = evaluate(W, X_test, y_test)
    print(acc)

(26000,)
Epochs : 1
it 0/260, loss = 5197.375000
it 200/260, loss = 3476.968964
Epochs : 2
it 0/260, loss = 3083.260594
it 200/260, loss = 2066.689532
Epochs : 3
it 0/260, loss = 1832.945955
it 200/260, loss = 1228.509405
Epochs : 4
it 0/260, loss = 1089.608695
it 200/260, loss = 730.446069
Epochs : 5
it 0/260, loss = 648.034061
it 200/260, loss = 435.120187
74.43076923076923
